In [1]:
import argparse
import sqlite3
import shutil
import time
import os
import pandas as pd
import numpy as np
import random

# ДЗ

Задание 1 [3 балла]. Создайте схему БД "Университет".

Она должна включать в себя минимально следующие отношения:

    - студенты (минимальные атрибуты: ФИО, номер зачётки, дата рождения, группа, год поступления)
    - посещаемые курсы (минимальные атрибуты: название курса, тип курса, дата введения в программу)
    - преподаватели (минимальные атрибуты: ФИО, дата рождения, дата начала работы в нашем университете, дата начала трудовой деятельности)
Атрибуты отношений должны быть осмысленными и соответствовать тематике.

Задание 2 [1 балл]. Заполните вашу схему фейковыми записями - в отношении Студенты должно быть не менее 150 строк

В остальных - не менее 20 строк.

Задание 3 [1 балл]. Выведите топ-10 курсов, которые ведут наименьшее число преподавателей. Расположите в порядке возрастания числа преподавателей.

Задание 4 [2 балла]. Напишите запрос, который выведет топ-10 преподавателей, которые преподают у наибольшего числа студентов. Расположите их в порядка убывания числа студентов.

Задание 5 [1.5 балла]. Для каждой группы студентов, поступивших не ранее 2019 года, выведите размах между максимальной и минимальной датами рождений студентов в этой группе.

Задание 6 [1.5 балла]. Среди преподавателей, родившихся в период между самой ранней датой начала работы в нашем университете и самой ранней датой начала трудовой деятельности по всем преподавателям, определите самое популярное имя.

### Задание 1 [3 балла]. 
Создайте схему БД "Университет".

In [2]:
conn = sqlite3.connect('university') 
c = conn.cursor()
c.execute("DROP TABLE IF EXISTS students")
c.execute("DROP TABLE IF EXISTS courses")
c.execute("DROP TABLE IF EXISTS teachers")
c.execute("DROP TABLE IF EXISTS classes")
c.execute("DROP TABLE IF EXISTS student_plans") 

### Задание 2 [1 балл].
Заполните вашу схему фейковыми записями - в отношении Студенты должно быть не менее 150 строк

В остальных - не менее 20 строк.

In [3]:
c.execute('''
          CREATE TABLE IF NOT EXISTS students
          ([id] INTEGER PRIMARY KEY, [full_name] TEXT, [gradebook] TEXT, [birth_date] DATE, [group_] TEXT, [enroll_year] INTEGER)
          ''')

In [4]:
q = '''
    INSERT INTO students (id, full_name, gradebook, birth_date, group_, enroll_year)

        VALUES
        
    '''
names = ('Александр,Марк,Мирон,Дмитрий,Степан,Максим,Михаил,Арсений,Всеволод,Иван,Антон,Даниил,Виктор,Тимофей,Давид,Демьян,Павел,Григорий,'+\
        'Ярослав,Кирилл,Алексей,Владислав,Виктория,Кира,Анна,Валерия,Нина,София,Арина,Сафия,Злата,Анастасия,Елизавета,Василиса,Алёна,Алина,'+\
        'Мирослава,Николь,Марина,Вера,Малика,Дарья,Диана,Евгения,Ариана,Мия,Софья,Антонина,Полина,Мария').split(',')
surnames = ('Смирнов,Иванов,Кузнецов,Соколов,Попов,Лебедев,Козлов,Новиков,Морозов,Петров,Волков,Соловьёв,Васильев,Зайцев,Павлов,Семёнов,Голубев,'+\
            'Виноградов,Богданов,Воробьёв,Фёдоров,Михайлов,Беляев,Тарасов,Белов,Комаров,Орлов,Киселёв,Макаров,Андреев,Ковалёв,Ильин,Гусев,Титов,'+\
            'Кузьмин,Кудрявцев,Баранов,Куликов,Алексеев,Степанов,Яковлев,Сорокин,Сергеев,Романов,Захаров,Борисов,Королёв,Герасимов,Пономарёв,'+\
            'Григорьев').split(',')
students = set()
ln = 10000
while(len(students)<ln):
    i = len(students)
    j = random.randint(0, len(names)-1)
    k = random.randint(0, len(surnames)-1)
    gradebook = str(i); gradebook = ("0"*(3-len(gradebook)))+gradebook
    gradebook = str(random.randint(0, 100)) + gradebook; gradebook = ("0"*(3*2-len(gradebook)))+gradebook
    birthyear = 1960+random.randint(0, 43)
    enroll_year = str(birthyear+17+random.randint(0,2))
    students.add((i+1, names[j] + " " + surnames[k] + ('a' if j>=22 else ''), gradebook, 
                       str(birthyear)+'-'\
                       "0" + str(random.randint(1,9))+'-' + str(random.randint(10,28)), 
                       enroll_year[-2:]+str(random.randint(1,4)),
                       enroll_year))
    
students = list(students)               
for i in range(len(students)):
    v = students[i]
    q += f"\t\t{str(v)}{'' if i==len(students)-1 else ','}\n"
# print(q)
c.execute(q)

In [5]:
c.execute('''
select * from students
'''
)
pd.DataFrame(c.fetchall(), columns=['id', 'full_name', 'gradebook', 'birth_date', 'group_', 'enroll_year']).reset_index(drop=True).set_index('id')

,full_name,gradebook,birth_date,group_,enroll_year
id,,,,,
1,Диана Тарасовa,049000,1986-06-25,041,2004
2,Вера Степановa,077001,2003-02-14,203,2020
3,Ярослав Ковалёв,071002,1978-04-28,972,1997
4,Николь Соколовa,039003,1995-05-18,132,2013
5,Кира Фёдоровa,051004,1965-02-19,844,1984
...,...,...,...,...,...
9996,Павел Морозов,899995,1977-06-12,963,1996
9997,Василиса Яковлевa,469996,1977-02-27,961,1996
9998,Марк Борисов,229997,1986-02-24,051,2005


In [6]:
c.execute('''
          CREATE TABLE IF NOT EXISTS courses
          ([id] INTEGER PRIMARY KEY, [name] TEXT, [type] TEXT, [date] DATE)
          ''')
q = '''
    INSERT INTO courses (id, name, type, date)

        VALUES
                
    '''

courses = [(1, 'Architecture', 'Arts and Humanities', '1960-01-01'),
            (2, 'Art', 'Arts and Humanities', '1960-01-02'),
            (3, 'Design', 'Arts and Humanities', '1960-01-03'),
            (4, 'History', 'Arts and Humanities', '1960-01-04'),
            (5, 'Marketing', 'Business and Management', '1960-01-05'),
            (6, 'Finance', 'Business and Management', '1960-01-06'),
            (7, 'Computer Science', 'Engineering and Technology', '1960-01-07'),
            (8, 'Chemical Engineering', 'Engineering and Technology', '1960-01-08'),
            (9, 'Engineering', 'Engineering and Technology', '1960-01-09'),
            (10, 'Agriculture', 'Life Sciences and Medicine', '1960-01-10'),
            (11, 'Biology', 'Life Sciences and Medicine', '1960-01-11'),
            (12, 'Dentistry', 'Life Sciences and Medicine', '1960-01-12'),
            (13, 'Medicine', 'Life Sciences and Medicine', '1960-01-13'),
            (14, 'Chemistry', 'Natural Sciences', '1960-01-14'),
            (15, 'Earth Sciences', 'Natural Sciences', '1960-01-15'),
            (16, 'Environmental Sciences', 'Natural Sciences', '1960-01-16'),
            (17, 'Geography', 'Natural Sciences', '1960-01-17'),
            (18, 'Accounting', 'Social Sciences', '1960-01-18'),
            (19, 'Antropology', 'Social Sciences', '1960-01-19'),
            (20, 'Media_Studies', 'Social Sciences', '1960-01-20')]              
for i in range(len(courses)):
    v = courses[i]
    q += f"\t\t{str(v)}{'' if i==len(courses)-1 else ','}\n"

# print(q)
c.execute(q)
c.execute('''
select * from courses
'''
)
pd.DataFrame(c.fetchall(), columns=['id', 'name', 'type', 'date']).reset_index(drop=True).set_index('id')

,name,type,date
id,,,
1,Architecture,Arts and Humanities,1960-01-01
2,Art,Arts and Humanities,1960-01-02
3,Design,Arts and Humanities,1960-01-03
4,History,Arts and Humanities,1960-01-04
5,Marketing,Business and Management,1960-01-05
6,Finance,Business and Management,1960-01-06
7,Computer Science,Engineering and Technology,1960-01-07
8,Chemical Engineering,Engineering and Technology,1960-01-08
9,Engineering,Engineering and Technology,1960-01-09


In [7]:
c.execute('''
          CREATE TABLE IF NOT EXISTS teachers
          ([id] INTEGER PRIMARY KEY, [name] TEXT, [birth_date] DATE, [enroll_date] DATE, [teach_date] DATE)
          ''')
q = '''
    INSERT INTO teachers (id, name, birth_date, enroll_date, teach_date)

        VALUES
        
    '''
names = ('Александр,Марк,Мирон,Дмитрий,Степан,Максим,Михаил,Арсений,Всеволод,Иван,Антон,Даниил,Виктор,Тимофей,Давид,Демьян,Павел,Григорий,'+\
        'Ярослав,Кирилл,Алексей,Владислав,Виктория,Кира,Анна,Валерия,Нина,София,Арина,Сафия,Злата,Анастасия,Елизавета,Василиса,Алёна,Алина,'+\
        'Мирослава,Николь,Марина,Вера,Малика,Дарья,Диана,Евгения,Ариана,Мия,Софья,Антонина,Полина,Мария').split(',')
surnames = ('Смирнов,Иванов,Кузнецов,Соколов,Попов,Лебедев,Козлов,Новиков,Морозов,Петров,Волков,Соловьёв,Васильев,Зайцев,Павлов,Семёнов,Голубев,'+\
            'Виноградов,Богданов,Воробьёв,Фёдоров,Михайлов,Беляев,Тарасов,Белов,Комаров,Орлов,Киселёв,Макаров,Андреев,Ковалёв,Ильин,Гусев,Титов,'+\
            'Кузьмин,Кудрявцев,Баранов,Куликов,Алексеев,Степанов,Яковлев,Сорокин,Сергеев,Романов,Захаров,Борисов,Королёв,Герасимов,Пономарёв,'+\
            'Григорьев').split(',')
teachers = set()
ln = 1000
while(len(teachers)<ln):
    i = len(teachers)
    j = random.randint(0, len(names)-1)
    k = random.randint(0, len(surnames)-1)
    birthyear = 1950+random.randint(0, 36)
    teach_year = birthyear+23+random.randint(0,6)
    teachers.add((i+1, names[j] + " " + surnames[k] + ('a' if j>=22 else ''), 
                       str(birthyear) + "-0" + str(random.randint(1,9)) + '-' + str(random.randint(10,28)),
                       str(teach_year+random.randint(1, 15)) + "-0" + str(random.randint(1,9)) + '-' + str(random.randint(10,28)),
                       str(teach_year) + "-0" + str(random.randint(1,9)) + '-' + str(random.randint(10,28))))
    
teachers = list(teachers)               
for i in range(len(teachers)):
    v = teachers[i]
    q += f"\t\t{str(v)}{'' if i==len(teachers)-1 else ','}\n"
# print(q)
c.execute(q)
c.execute('''
select * from teachers
'''
)
pd.DataFrame(c.fetchall(), columns=['id', 'name', 'birth_date', 'enroll_date', 'teach_date']).reset_index(drop=True).set_index('id')

,name,birth_date,enroll_date,teach_date
id,,,,
1,Анна Зайцевa,1954-04-16,1996-05-21,1981-04-15
2,Арсений Богданов,1955-03-28,1991-03-14,1980-06-17
3,Мия Павловa,1977-06-23,2009-08-26,2003-03-10
4,Ариана Гусевa,1977-04-11,2010-04-17,2001-07-19
5,Тимофей Орлов,1952-01-22,1978-07-22,1975-01-20
...,...,...,...,...
996,Кирилл Белов,1950-06-12,1982-09-27,1977-01-20
997,Виктор Комаров,1968-09-14,2007-06-12,1993-05-17
998,Степан Воробьёв,1957-07-17,1987-08-11,1986-03-11


In [8]:
c.execute('''
          CREATE TABLE IF NOT EXISTS classes
          ([id] INTEGER PRIMARY KEY, [name_id] INTEGER, [group_] TEXT, [teacher_id] INTEGER)
          ''')
q = '''
    INSERT INTO classes (id, name_id, group_, teacher_id)

        VALUES
        
    '''

classes = set()
ln = 200
while(len(classes)<ln):
    i = len(classes)
    name_id = random.randint(1, len(courses))
    _year = random.randint(1960, 2022)
    group_ = str(_year)[-2:] + str(random.randint(1, 4))
    candidates = [i+1 for i in range(len(teachers)) if int(teachers[i][3][:4])<_year]
    if(len(candidates)==0): continue
    teacher_id = np.random.choice(candidates, 1)[0]
    classes.add((i+1, name_id, group_, teacher_id))
    
classes = list(classes)               
for i in range(len(classes)):
    v = classes[i]
    q += f"\t\t{str(v)}{'' if i==ln-1 else ','}\n"
# print(q)
c.execute(q)
c.execute('''
select * from classes
'''
)
pd.DataFrame(c.fetchall(), columns=['id', 'name_id', 'group_', 'teacher_id',]).reset_index(drop=True).set_index('id')

,name_id,group_,teacher_id
id,,,
1,3,151,929
2,17,193,181
3,14,831,329
4,18,071,523
5,11,213,789
...,...,...,...
196,3,922,882
197,4,784,422
198,14,941,878


In [9]:
c.execute('''
          CREATE TABLE IF NOT EXISTS student_plans
          ([id] INTEGER PRIMARY KEY, [student_id] INTEGER, [class_id] INTEGER)
          ''')
q = '''
    INSERT INTO student_plans (id, student_id, class_id)

        VALUES
        
    '''

student_plans = []
for j in range(len(students)):
    group_ = students[j][4]
    for course_id in np.random.choice(range(len(courses)), 10):
        i = len(student_plans)
        ind=-1
        for k in range(len(classes)):
            if(classes[k][1]==course_id and classes[k][2]==group_):
                ind=k
                break
        if(ind>=0): student_plans.append((i+1, j+1, ind+1))

for i in range(len(student_plans)):
    v = student_plans[i]
    q += f"\t\t{str(v)}{'' if i==len(student_plans)-1 else ','}\n"
# print(q)
c.execute(q)
c.execute('''
select * from student_plans
'''
)
pd.DataFrame(c.fetchall(), columns=['id', 'student_id', 'class_id']).reset_index(drop=True).set_index('id')

,student_id,class_id
id,,
1,4,161
2,11,197
3,11,67
4,12,198
5,12,198
...,...,...
4903,9988,29
4904,9993,42
4905,9998,62


### Задание 3 [1 балл].
Выведите топ-10 курсов, которые ведут наименьшее число преподавателей. Расположите в порядке возрастания числа преподавателей.

In [10]:
c.execute('''
select courses.id as course_id, courses.name as course_name, COUNT(DISTINCT(teacher_id)) as teacher_count from courses
    join classes on (classes.name_id=courses.id)
    group by course_name
    order by teacher_count ASC
    limit 10
'''
)
pd.DataFrame(c.fetchall(), columns=['course_id', 'course_name', 'teachers_count']).reset_index(drop=True).set_index('course_id')

,course_name,teachers_count
course_id,,
2,Art,4
9,Engineering,4
10,Agriculture,5
11,Biology,6
5,Marketing,6
17,Geography,7
18,Accounting,9
1,Architecture,9
13,Medicine,9


### Задание 4

In [11]:
c.execute('''
select teachers.id as teacher_id, teachers.name as teacher_name, t1.student_count from teachers
    left join (
        select teacher_id, COUNT(DISTINCT(student_id)) as student_count from student_plans
            left join classes on (student_plans.class_id=classes.id)
        group by teacher_id) as t1
    on (teachers.id=t1.teacher_id)
    order by student_count DESC
    limit 10
'''
)
pd.DataFrame(c.fetchall(), columns=['teacher_id', 'teacher_name', 'student_count']).reset_index(drop=True).set_index('teacher_id')

,teacher_name,student_count
teacher_id,,
127,Мирон Ильин,160
434,Всеволод Романов,129
548,Марина Богдановa,96
962,Софья Беловa,76
422,Михаил Киселёв,68
71,Иван Титов,62
170,Дарья Смирновa,62
79,Кира Яковлевa,61
30,Алина Смирновa,58


### Задание 5 [1.5 балла]. 
Для каждой группы студентов, поступивших не ранее 2019 года, выведите размах между максимальной и минимальной датами рождений студентов в этой группе.

In [12]:
c.execute('''
select group_, MAX(birth_date) - MIN(birth_date) from students
    where (group_>"190" and group_<"240")
    group by group_
'''
)
pd.DataFrame(c.fetchall(), columns=['group', 'years_difference']).reset_index(drop=True).set_index('group')

,years_difference
group,
191,2
192,2
193,2
194,2
201,2
202,2
203,2
204,2
211,1


### Задание 6 [1.5 балла]. 
Среди преподавателей, родившихся в период между самой ранней датой начала работы в нашем университете и самой ранней датой начала трудовой деятельности по всем преподавателям, определите самое популярное имя.

In [13]:
c.execute('''
select top_name from (
    select _name as top_name, COUNT(*) as cnt from (
        select SUBSTRING(name, 1, INSTR(name, ' ') - 1) as _name from teachers
            join (
                select MIN(teach_date) as td, MIN(enroll_date) as ed from teachers
            ) 
            where (birth_date BETWEEN td and ed)
        )
    group by _name
    order by cnt DESC
    )
limit 1
'''
)
pd.DataFrame(c.fetchall(), columns=['top name']).reset_index(drop=True)

,top name
0,Алексей
